In [15]:
import numpy as np
import os
# import face_recognition
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model,save_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import gc

In [16]:
face_net=cv2.dnn.readNetFromCaffe('deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel')
# face_net=cv2.dnn.readNetFromCaffe('deploy.prototxt', 'weights.caffemodel')
genders= ['Male','Female']

In [17]:
def load_data(data_dir):
    images=[]
    labels=[]
    c=0
    # output=r'C:\Users\hardi\VS Code\ML\cropped'
    for filename in os.listdir(data_dir):
        if filename.endswith('.jpg'):
            gender = int(filename.split('_')[1])  # 0 for male, 1 for female
            img_path=os.path.join(data_dir,filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (100, 100))
            # cv2.imwrite(output+str(f'\{c} {gender}.jpg'),img)
            # c+=1
            labels.append(gender)
            images.append(img)
    # labels=np.array(labels)
    # images=np.array(images,dtype=np.float32)/255.0

    return images,labels

In [18]:
def load_preprocess_data(data_dir,batch_size,limit):
    images=[]
    labels=[]
    i=0
    for filename in os.listdir(data_dir):
        img=cv2.imread(os.path.join(data_dir,filename))
        if img is None:
            print(f"Failed to load image:{filename}")
            continue
        (h, w) = img.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        face_net.setInput(blob)
        detections = face_net.forward()
        max_=0
        face=None
        for j in range(detections.shape[2]):
            confidence = detections[0, 0, j, 2]
            if(confidence>=max_):
                max_=confidence
                face_box = detections[0, 0, j, 3:7] * np.array([w, h, w, h])
                (start_X, start_Y, end_X, end_Y) = face_box.astype("int")
                start_X = max(0, start_X)
                start_Y = max(0, start_Y)
                end_X = min(w, end_X)
                end_Y = min(h, end_Y)
                face=img[start_Y:end_Y,start_X:end_X]
        if max_>0.4 and face is not None:
            h,w=face.shape[:2]
            face=face[:h//2,:]
            face=cv2.resize(face,(100,100))
            images.append(face)
            gender = int(filename.split('_')[1])  # 0 for male, 1 for female
            labels.append(gender)
        else:
            print(f'No Face Detected in {filename}')
        i+=1
        if(i%batch_size==0):
            print(f'Processed {i} images')
            yield np.array(images)/255.0,np.array(labels)
            images,labels=[],[]
            gc.collect()

        if limit and i>=limit:
            break
    if images:
        yield np.array(images)/255.0,np.array(labels)
        gc.collect()
    # images=np.array(images)/255.0
    # labels=np.array(labels)

    

In [19]:
data_dir='.\dataset'

images=[]
labels=[]

for image,label in load_preprocess_data(data_dir,batch_size=100,limit=24000):
    images.append(image)
    labels.append(label)

images=np.vstack(images)
labels=np.concatenate(labels)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\hardi\AppData\Local\Temp\ipykernel_14352\4168312264.py:1: SyntaxWarning: invalid escape sequence '\d'
  data_dir='.\dataset'


No Face Detected in 105_1_0_20170112213021902.jpg
Processed 100 images
Processed 200 images
Processed 300 images
Processed 400 images
Processed 500 images
Processed 600 images
Processed 700 images
No Face Detected in 15_1_0_20170109204416506.jpg
Processed 800 images
Processed 900 images
Processed 1000 images
Processed 1100 images
No Face Detected in 17_1_0_20170109205309078.jpg
Processed 1200 images
Processed 1300 images
Processed 1400 images
Processed 1500 images
Processed 1600 images
No Face Detected in 1_0_0_20170109194225586.jpg
Processed 1700 images
Processed 1800 images
Processed 1900 images
No Face Detected in 1_0_2_20161219195311539.jpg
Processed 2000 images
Processed 2100 images
Processed 2200 images
Processed 2300 images
No Face Detected in 1_1_0_20161219194707290.jpg
Processed 2400 images
Processed 2500 images
No Face Detected in 1_1_2_20161219160842277.jpg
Processed 2600 images
Processed 2700 images
Processed 2800 images
Processed 2900 images
Processed 3000 images
Processed

In [20]:
# plt.imshow(cv2.cvtColor(images[500], cv2.COLOR_BGR2RGB))
# plt.show()
X_train, X_val, y_train, y_val = train_test_split(images,labels, test_size=0.2, random_state=42)
# print(X_train.shape(),y_train.shape())

In [21]:
# Define the model
gender_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100,100,3)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    # BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(128, (3, 3), activation='relu'),
    # MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: Male (0) or Female (1)
])

C:\Users\hardi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
gender_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
#Model Training
# gender_model.fit(datagen.flow(X_train, y_train, batch_size=80), epochs=20, validation_data=(X_val, y_val))
gender_model.fit(X_train, y_train, batch_size=100, epochs=15, validation_data=(X_val, y_val))
gender_model.save('Gender_Model.keras')

Epoch 1/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 46s 227ms/step - accuracy: 0.5698 - loss: 0.7496 - val_accuracy: 0.7265 - val_loss: 0.5382
Epoch 2/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 42s 218ms/step - accuracy: 0.7342 - loss: 0.5263 - val_accuracy: 0.7688 - val_loss: 0.4588
Epoch 3/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 42s 216ms/step - accuracy: 0.7779 - loss: 0.4651 - val_accuracy: 0.7930 - val_loss: 0.4278
Epoch 4/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 209ms/step - accuracy: 0.7992 - loss: 0.4274 - val_accuracy: 0.8013 - val_loss: 0.4174
Epoch 5/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 211ms/step - accuracy: 0.8101 - loss: 0.4058 - val_accuracy: 0.8138 - val_loss: 0.3914
Epoch 6/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 208ms/step - accuracy: 0.8227 - loss: 0.3835 - val_accuracy: 0.8174 - val_loss: 0.3832
Epoch 7/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 206ms/step - accuracy: 0.8379 - loss: 0.3585 - val_accuracy: 0.8297 - val_loss: 0.3816
Epoch 8/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 44s 229ms/step - accuracy: 0.8488 - loss: 0

In [61]:
gender_model=load_model('Gender_Model.keras')

In [62]:
def extract_features(image, face_box):
    (startX, startY, endX, endY) = face_box
    face = image[startY:endY, startX:endX]
    
    # Extracting eye and forehead regions by assuming that it would occupy the upper half of the face_box
    height, width = face.shape[:2]
    eyes_region = face[:height//2, :]
    # Resizing the eyes_region to match the input size of the gender model
    eyes_region_resized = cv2.resize(eyes_region, (100,100))
    # plt.imshow(cv2.cvtColor(eyes_region_resized, cv2.COLOR_BGR2RGB))
    # plt.show()
    
    return eyes_region_resized


In [63]:
def predict_gender(image):
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_net.setInput(blob)
    detections = face_net.forward()
    max=0
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if(confidence>=max):
            max=confidence
            face_box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (start_X, start_Y, end_X, end_Y) = face_box.astype("int")
            features = extract_features(image,(start_X, start_Y, end_X, end_Y))
            features = np.expand_dims(features, axis=0)  # Add batch dimension for the input to work with neural network
            features = features / 255.0  # Normalize pixel values
            gender_preds=gender_model.predict(features)   
            if(gender_preds[0]<0.5):
                gender=genders[0]
            else:
                gender=genders[1]

    return gender_preds,gender


In [64]:
# image = cv2.imread(r'C:\Users\hardi\VS Code\ML\RWMFD_part_2_pro\00111\003.jpg')
image = cv2.imread(r"C:\Users\hardi\VS Code\ML\WhatsApp Image 2024-05-23 at 09.46.30_11d6bd36.jpg")
x,result = predict_gender(image)
print(x,result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
[[0.99629086]] Female


In [66]:
def pred_accuracy(data_dir):
    images=[]
    test_label=[]
    pred_label=[]
    for filename in os.listdir(data_dir):
        img=cv2.imread(os.path.join(data_dir,filename))
        if img is None:
            print(f"Failed to load image:{filename}")
            continue
        else:
            images.append(img)
            gender=int(filename[-5])
            test_label.append(gender)
    test_label=np.array(test_label)
    i=0
    cost=0
    for img in images:
        x,gender=predict_gender(img)
        if(gender=='Male'):
            pred_label.append(0)
        else:
            pred_label.append(1)
    pred_label=np.array(pred_label)
    accuracy = np.mean(pred_label == test_label)

    return accuracy

In [70]:
test_dir='.\Cropped'
print(f'Accuracy: {pred_accuracy(test_dir)}')

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\hardi\AppData\Local\Temp\ipykernel_14352\3753744714.py:1: SyntaxWarning: invalid escape sequence '\C'
  test_dir='.\Cropped'


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━

In [71]:
import gradio as gr

def predict_gen(img):
    x,result = predict_gender(img)
    return result

interface = gr.Interface(fn=predict_gen, inputs="image", outputs="text")
interface.launch()


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
